In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parameter as P
import torch.optim as optim

import numpy as np

In [2]:
userIds = set()
movieIds = set()
triplets = []

file1 = open('../../ml-100k/u.data', 'r')
for line in file1.readlines():
    uid, mid, rating, timestamp = line.split('	')
    userIds.add(int(uid))
    movieIds.add(int(mid))
    triplets.append([uid,mid,rating])

triplets = np.array(triplets, dtype='int')
numUsers = max(userIds)
numItems = max(movieIds)
X = triplets[:,0:2]
Y = triplets[:,2]

trainX = X[:-10000]
valX = X[-10000:]
trainY = Y[:-10000]
valY = Y[-10000:]

In [8]:
class FCMF(nn.Module):
    '''
    Base class for Fully-Connected Matrix Factorization networks
    '''
    
    def __init__ (self, N, M, D, D_, K, layers):
        '''
        variable definitions taken from paper: https://arxiv.org/pdf/1511.06443.pdf
        
        @param N:  Number of users
        @param M:  Number of items
        @param D:  size of latent-feature vectors
        @param D_: num rows in latent-features matrices
        @param K:  num cols in latent-feature matrices
        
        @param layers: list of hidden layer sizes; does not include input or output
        '''
        
        assert (min(N,M,D,D_,K) > 0), "Params must be nonzero and positive"
        assert (len(layers) > 0),     "Must have nonzero hidden layers"
        
        ########################################################################
        
        super(FCMF, self).__init__()
        
        self.N, self.M, self.D, self.D_, self.K = N, M, D, D_, K
        
        self.userLatentVectors = P.Parameter(torch.rand(N,D, requires_grad=True))
        self.itemLatentVectors = P.Parameter(torch.rand(M,D, requires_grad=True))
        
        self.userLatentMatrices = P.Parameter(torch.rand(N,D_,K, requires_grad=True))
        self.itemLatentMatrices = P.Parameter(torch.rand(M,D_,K, requires_grad=True))
        
        linear_inputs = [2*D + D_] + layers
        linear_outputs = layers + [1]
        
        self.layers = nn.ModuleList([nn.Linear(i,o) for (i,o) in zip(linear_inputs, linear_outputs)])
        
    def forward(self, x):
        '''
        @param x: let this be a tensor of size (X, 2): (user index, item index)
        
        WARNING: 
            - forward currently does not account for user/items outside of training data
            - mitigations include returning smart averages    
        '''        
        userIndices, itemIndices = x[:,0].long(), x[:,1].long()
                
        userLatMats = self.userLatentMatrices[userIndices]
        itemLatMats = self.itemLatentMatrices[itemIndices]
        latentDotProducts = torch.sum(userLatMats * itemLatMats, dim=-1)        
        
        x = torch.hstack([
            self.userLatentVectors[userIndices],
            self.itemLatentVectors[itemIndices],
            latentDotProducts
        ])
        
        for l in self.layers[:-1]:
            x = F.relu(l(x))
        
        # TODO: should last layer go through a sigmoid?
        out = self.layers[-1](x)
        return out
    
    def gradAll(self):
        self._setGrads(True, True, True, True, True)
    
    def gradNetwork(self):
        self._setGrads(False, False, False, False, True)

    def gradLatent(self):
        self._setGrads(True, True, True, True, False)

    def _setGrads(self, userVec, itemVec, userMat, itemMat, net):
        self.userLatentVectors.requires_grad_(userVec)
        self.itemLatentVectors.requires_grad_(itemVec)
        
        self.userLatentMatrices.requires_grad_(userMat)
        self.itemLatentMatrices.requires_grad_(itemMat)
        
        self.layers.requires_grad = net
        

In [63]:
def getBatches(X, Y, usersPerBatch=100):
    '''
    batchSize = min(N - start, usersPerBatch) * M
    '''
    N = X.shape[0]
    
    start = 0
    while start < N:

        if start+usersPerBatch + 1 < N:
            batch_x = torch.tensor(X[start:start+usersPerBatch]).to(device)
            batch_y = torch.tensor(Y[start:start+usersPerBatch]).to(device).float()
            start += usersPerBatch
            yield (batch_x, batch_y)

        else:
            batch_x = torch.tensor(X[start:]).to(device)
            batch_y = torch.tensor(Y[start:]).to(device).float()
            start += usersPerBatch
            yield (batch_x, batch_y)
        
    

def trainEpoch(opt, criterion, model, X,Y):
    model.train()
    opt.zero_grad()
    totalLoss = 0
    for batch_x, batch_y in getBatches(X,Y):
        optimizer.zero_grad()
        pred_y = model(batch_x)
        loss = criterion(batch_y, pred_y)
        totalLoss += loss
        loss.backward()
        optimizer.step()
    return totalLoss/X.shape[0]

def evaluate(criterion, model, X,Y):
    model.eval()
    loss = 0
    accuracy = 0
    for batch_x, batch_y in getBatches(X,Y):
        pred_y = model(batch_x)
        loss += criterion(batch_y, pred_y)
        matching = (torch.round(pred_y.detach()).flatten() == batch_y.flatten()).type(torch.uint8).sum()
        matching = matching
        print(matching)
        accuracy += matching
    return loss/X.shape[0], accuracy/X.shape[0]


In [64]:
# device = 'cuda'
device = 'cpu'

In [65]:
fc3 = FCMF(numUsers+1, numItems+1 ,10,10,5,[5]).to(device)
# Paper uses RMSE as objective and RMSProp optimizer
criterion = nn.MSELoss()
optimizer = optim.RMSprop(fc3.parameters(), lr=0.0001)

epochs = 0
max_epochs = 50
while epochs < max_epochs:
    fc3.gradAll()
    loss = trainEpoch(optimizer, criterion, fc3, trainX,trainY)
    # fc3.gradLatent()
    # loss += trainEpoch(optimizer, criterion, fc3, trainX,trainY)
    # fc3.gradNetwork()
    # loss += trainEpoch(optimizer, criterion, fc3, trainX,trainY)
    loss = loss
    val_loss, val_acc = evaluate(criterion, fc3, valX, valY)
    print("Epoch {} Train Loss: {} Val Loss: {} Val Acc: {}".format(epochs,loss, val_loss, val_acc))
    epochs+=1


tensor(5)
tensor(4)
tensor(3)
tensor(4)
tensor(5)
tensor(9)
tensor(4)
tensor(6)
tensor(2)
tensor(4)
tensor(3)
tensor(5)
tensor(3)
tensor(13)
tensor(11)
tensor(7)
tensor(5)
tensor(6)
tensor(10)
tensor(2)
tensor(3)
tensor(6)
tensor(7)
tensor(3)
tensor(3)
tensor(8)
tensor(5)
tensor(4)
tensor(2)
tensor(5)
tensor(4)
tensor(4)
tensor(4)
tensor(8)
tensor(4)
tensor(7)
tensor(7)
tensor(9)
tensor(7)
tensor(4)
tensor(7)
tensor(7)
tensor(13)
tensor(5)
tensor(4)
tensor(8)
tensor(7)
tensor(6)
tensor(5)
tensor(2)
tensor(9)
tensor(3)
tensor(2)
tensor(6)
tensor(3)
tensor(4)
tensor(11)
tensor(4)
tensor(6)
tensor(7)
tensor(6)
tensor(5)
tensor(7)
tensor(4)
tensor(6)
tensor(7)
tensor(7)
tensor(5)
tensor(7)
tensor(7)
tensor(8)
tensor(7)
tensor(10)
tensor(4)
tensor(8)
tensor(2)
tensor(6)
tensor(12)
tensor(11)
tensor(10)
tensor(3)
tensor(9)
tensor(4)
tensor(8)
tensor(3)
tensor(6)
tensor(8)
tensor(5)
tensor(6)
tensor(2)
tensor(9)
tensor(7)
tensor(3)
tensor(5)
tensor(7)
tensor(5)
tensor(3)
tensor(7)
tensor(10)


KeyboardInterrupt: 